These algorithms are useful in the case of searching a string within another string.

Pattern searching is an important problem in computer science. When we do search for a string in notepad/word file or browser or database, pattern searching algorithms are used to show the search results.

# Naive algorithm for Pattern Searching

Slide the pattern over text one by one and check for a match. If a match is found, then slides by 1 again to check for subsequent matches.

In [2]:
def search(a,b):
    n=len(a)
    m=len(b)
    result=[]
    for i in range(n-m+1):
        j=0
        while j<m:
            if a[i+j]!=b[j]:
                break
            j+=1
        if j==m:
            result.append(i)
    print(f"The pattern was found at {' '.join(map(str,result))}")


if __name__ == '__main__':
    a="AABAACAADAABAAABAA"
    b="AABA"
    search(a,b)


The pattern was found at 0 9 13


<pre>
<strong>What is the best case?</strong>
The best case occurs when the first character of the pattern is not present in text at all.

txt[] = "AABCCAADDEE"; 
pat[] = "FAA";
The number of comparisons in best case is O(n).

<strong>What is the worst case ?</strong>
The worst case of Naive Pattern Searching occurs in following scenarios.
1) When all characters of the text and pattern are same.

txt[] = "AAAAAAAAAAAAAAAAAA"; 
pat[] = "AAAAA";
2) Worst case also occurs when only the last character is different.

txt[] = "AAAAAAAAAAAAAAAAAB"; 
pat[] = "AAAAB";

The number of comparisons in the worst case is O(m*(n-m+1)). Although strings which have repeated characters are not likely to appear in English text, they may well occur in other applications (for example, in binary texts).
</pre>

# Optimized Naive Algorithm for Pattern Searching

In the original Naive String matching algorithm , we always slide the pattern by 1. When all characters of pattern are different, we can slide the pattern by more than 1. Let us see how can we do this. When a mismatch occurs after j matches, we know that the first character of pattern will not match the j matched characters because all characters of pattern are different. So we can always slide the pattern by j without missing any valid shifts.

In [4]:
def search(text,pattern):
    n=len(text)
    m=len(pattern)
    result=[]
    i=0
    while i<=n-m:
        j=0
        while j<m:
            if text[i+j]!=pattern[j]:
                break
            j+=1
        if j==m:
            result.append(i)
            # print("pattern found at",i)
            i+=m
        elif j==0:
            i+=1
        else:
            i+=j
    print(f"The pattern was found at {' '.join(map(str,result))}")

if __name__ == '__main__':
    text="ABCEABCDABCEABCD"
    pattern="ABCD"
    search(text,pattern)


The pattern was found at 4 12


# KMP Algorithm for Pattern Searching

In [5]:
def processPattern(pattern):
    prefixArray=[0]*len(pattern)
    n=len(pattern)
    i=1
    j=0
    while i<n:
        if pattern[i]==pattern[j]:
            prefixArray[i]=j+1
            i+=1
            j+=1
        elif j==0:
            i+=1
        else:
            j=prefixArray[j-1]
    return prefixArray

def search(text,pattern):
    prefixArray=processPattern(pattern)
    n=len(text)
    m=len(pattern)
    result=[]
    i=0;j=0
    while i<n:
        if text[i]==pattern[j]:
            i+=1
            j+=1
            if j==m:
                # print("pattern found at ",i-m)
                result.append(i-m)
                j=prefixArray[j-1]
        elif j==0:
            i+=1
        else:
            j=prefixArray[j-1]
    print(f"The pattern was found at {','.join(map(str,result))}")

if __name__ == '__main__':
    # pattern=['a','a','b','a','a','b','a','a','a',]
    # processPattern(pattern)
    # text="abxabcabcaby"
    # pattern="abcaby"
    text="AABAACAADAABAABAA"
    pattern="AABA"
    search(text,pattern)


The pattern was found at 0,9,12


# Rabin-Karp Algorithm for Pattern Searching

<pre>
Like the Naive Algorithm, Rabin-Karp algorithm also slides the pattern one by one. But unlike the Naive algorithm, Rabin Karp algorithm matches the hash value of the pattern with the hash value of current substring of text, and if the hash values match then only it starts matching individual characters. So Rabin Karp algorithm needs to calculate hash values for following strings.



1) Pattern itself.
2) All the substrings of text of length m.

Since we need to efficiently calculate hash values for all the substrings of size m of text, we must have a hash function which has following property.
Hash at the next shift must be efficiently computable from the current hash value and next character in text or we can say hash(txt[s+1 .. s+m]) must be efficiently computable from hash(txt[s .. s+m-1]) and txt[s+m] i.e., hash(txt[s+1 .. s+m])= rehash(txt[s+m], hash(txt[s .. s+m-1])) and rehash must be O(1) operation.
</pre>

In [2]:
def search(text,pattern):
    n=len(text)
    m=len(pattern)
    q=101
    result=[]
    patHash=calculateHash(pattern,0,m,q)
    textHash=calculateHash(text,0,m,q)
    # print(textHash)
    # print(patHash)
    for i in range(1,n-m+2):
        # print(patHash,textHash)
        if patHash==textHash and checkEqual(text,i-1,i+m-1,pattern,0,m):
            result.append(i-1)
        if i<n-m+1:
            textHash=recalculateHash(text,i-1,i+m-1,textHash,q,m)
    print(f"The pattern was found at {','.join(map(str,result))}")

def recalculateHash(text,outgoing,incoming,oldHash,prime,m):
    newHash=oldHash-ord(text[outgoing])
    newHash=newHash//prime
    newHash+=ord(text[incoming])*(prime**(m-1))
    return newHash

def checkEqual(text,startText,endText,pattern,startPattern,endPattern):
    while startText<endText and startPattern<endPattern:
        # print(text[startText],pattern[startPattern])
        if text[startText]!=pattern[startPattern]:
            return False
        startText+=1
        startPattern+=1
    return True

def calculateHash(s,start,end,prime):
    hash=0
    while start<end:
        hash+=(ord(s[start]))*(prime**start)
        start+=1
    return hash

if __name__ == '__main__':
    text="AABAACAADAABAABA"
    pattern="AABA"
    # text="abedabc"
    # pattern="abc"
    search(text,pattern)


The pattern was found at 0,9,12


Time Complexity O(mn) if the hash function is bad, so it will start matching for all the hashes

# Boyer Moore Algorithm

Boyer Moore algorithm also preprocesses the pattern.
Boyer Moore is a combination of following two approaches.

1) Bad Character Heuristic

2) Good Suffix Heuristic

The Boyer Moore algorithm does preprocessing for the same reason. It processes the pattern and creates different arrays for both heuristics. At every step, it slides the pattern by the max of the slides suggested by the two heuristics. So it uses best of the two heuristics at every step.
Unlike the previous pattern searching algorithms, Boyer Moore algorithm starts matching from the last character of the pattern.

<strong>Bad Character Heuristic</strong>

The idea of bad character heuristic is simple. The character of the text which doesn’t match with the current character of the pattern is called the Bad Character. Upon mismatch, we shift the pattern until –

1) The mismatch becomes a match

2) Pattern P move past the mismatched character.

<strong>Case 1 – Mismatch become match</strong>

We will lookup the position of last occurrence of mismatching character in pattern and if mismatching character exist in pattern then we’ll shift the pattern such that it get aligned to the mismatching character in text T.

<strong>Case 2 – Pattern move past the mismatch character</strong>

We’ll lookup the position of last occurrence of mismatching character in pattern and if character does not exist we will shift pattern past the mismatching character

In the following implementation, we preprocess the pattern and store the last occurrence of every possible character in an array of size equal to alphabet size. If the character is not present at all, then it may result in a shift by m (length of pattern). Therefore, the bad character heuristic takes O(n/m) time in the best case.

In [3]:
def computeBadHeuristic(pattern,m):
    badChar=[-1]*256
    for i in range(m):
        badChar[ord(pattern[i])]=i  #last occurence
    return badChar

def search(text,pattern):
    n=len(text)
    m=len(pattern)
    badChar=computeBadHeuristic(pattern,m)
    i=0
    result=[]
    while i<=n-m:
        j=m-1 #checking from behind
        while j>=0 and text[i+j]==pattern[j]:
            j-=1
        if j==-1:  #if pattern matches
            result.append(i)
            if i+m<n:
                i+=m-badChar[ord(text[i+m])] #number of shifts
            else:
                i+=1
        else:
            i+=max(1,j-badChar[ord(text[i+j])]) #1 because we may get a -ve shift if the last occurence is in the right
    print(f"The pattern was found at {','.join(map(str,result))}")

if __name__ == '__main__':
    text="AABAACAADAABAABA"
    pattern="AABA"
    # text="ABAAABCD"
    # pattern="ABC"
    search(text,pattern)


The pattern was found at 0,9,12


The Bad Character Heuristic may take O(mn) time in worst case. The worst case occurs when all characters of the text and pattern are same. For example, txt[] = “AAAAAAAAAAAAAAAAAA” and pat[] = “AAAAA”